<h1>Getting Started</h1>
1. signup for Rasgo (https://app.rasgoml.com/account/register) and get your API key from the top right corner in the UI<br>
2. install pyrasgo (next cell down)<br>
3. check out the docs: https://docs.rasgoml.com/rasgo-docs/pyrasgo/user-defined-transforms-udts<br>
4. follow along with this tutorial

In [5]:
#!pip install pyrasgo --upgrade
import pyrasgo

In [11]:
#use your rasgo API key to connect
api_key = ''
rasgo = pyrasgo.connect(api_key)

In [12]:
# Get all available transforms
# Best place to see more info is https://docs.rasgoml.com/rasgo-docs/pyrasgo/user-defined-transforms-udts
transforms = rasgo.get.transforms()
for t in transforms:
    args = []
    for a in t.arguments:
        args.append(a['name'])
    print(t.id, t.name, '({})'.format(', '.join(args)))

170 rename (col_list, new_names)
178 dedupe (natural_key, order_method, order_col)
162 binning (binning_type, bucket_count, col_to_bucket)
179 union (source_id, union_all)
160 train_test_split (order_by, train_percent)
158 concat (concat_list)
161 to_date (date_columns, format_expression)
163 cast (col_list, type_list)
174 unpivot (value_column, name_column, column_list_vals)
168 levenshtein (columns1, column2)
172 drop_columns (include_cols, exclude_cols)
159 find_and_replace (replace_dict)
155 one_hot_encode (column)
156 datetrunc (date_columns, date_part)
165 math (math_ops)
173 filter (filter_statements)
169 moving_avg (input_columns, window_sizes, order_by, partition)
175 pivot (dimensions, pivot_column, value_column, agg_method, list_of_vals)
176 join (join_table_id, source_columns, join_type, joined_colums)
157 datepart (date_columns, date_part)
177 aggregate (group_items, aggregations)
166 substring (target_col, start_pos, end_pos)
164 impute (imputations, flag_missing_vals)
18

In [26]:
#sources = rasgo.get.data_sources()
#for source in sources:
#    if 'ADVENTUREWORKS' in source.name:
#        print(source.id, source.table)

In [18]:
# get some rasgo data sources to operate on
internet_sales = rasgo.get.data_source(id=2637)
customer = rasgo.get.data_source(id=2616)
geo = rasgo.get.data_source(id=2619)

<h2>Challenge 1: Cluster AW Customers by Attributes and Activity</h2>
<h3>...using only Rasgo UDTs and sklearn kmeans</h3>

In [21]:
# preview data set by generating a df and then previewing it with .head()
rasgo.read.source_data(customer.id, limit=100).head()

,CUSTOMERKEY,GEOGRAPHYKEY,CUSTOMERALTERNATEKEY,TITLE,FIRSTNAME,MIDDLENAME,LASTNAME,NAMESTYLE,BIRTHDATE,MARITALSTATUS,...,ENGLISHOCCUPATION,SPANISHOCCUPATION,FRENCHOCCUPATION,HOUSEOWNERFLAG,NUMBERCARSOWNED,ADDRESSLINE1,ADDRESSLINE2,PHONE,DATEFIRSTPURCHASE,COMMUTEDISTANCE
0,11000,26,AW00011000,None,Jon,V,Yang,0,1971-10-06,M,...,Professional,Profesional,Cadre,1,0,3761 N. 14th St,None,1 (11) 500 555-0162,2011-01-19,1-2 Miles
1,11001,37,AW00011001,None,Eugene,L,Huang,0,1976-05-10,S,...,Professional,Profesional,Cadre,0,1,2243 W St.,None,1 (11) 500 555-0110,2011-01-15,0-1 Miles
2,11002,31,AW00011002,None,Ruben,None,Torres,0,1971-02-09,M,...,Professional,Profesional,Cadre,1,1,5844 Linden Land,None,1 (11) 500 555-0184,2011-01-07,2-5 Miles
3,11003,11,AW00011003,None,Christy,None,Zhu,0,1973-08-14,S,...,Professional,Profesional,Cadre,0,1,1825 Village Pl.,None,1 (11) 500 555-0162,2010-12-29,5-10 Miles
4,11004,19,AW00011004,None,Elizabeth,None,Johnson,0,1979-08-05,S,...,Professional,Profesional,Cadre,1,4,7553 Harness Circle,None,1 (11) 500 555-0131,2011-01-23,1-2 Miles


In [22]:
# join tables to create initial data set

t0 = internet_sales.transform(
    transform_name='join',
    join_table_id=customer.id,
    join_type='LEFT',
    source_columns=['CUSTOMERKEY'],
    joined_colums=['CUSTOMERKEY']
)

#print(t0.preview_sql())

In [23]:
t0.preview()

,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,EXTENDEDAMOUNT,PRODUCTKEY,SALESORDERNUMBER,SALESAMOUNT,...,TITLE,MARITALSTATUS,SPANISHEDUCATION,TOTALCHILDREN,LASTNAME,YEARLYINCOME,FIRSTNAME,FRENCHOCCUPATION,GEOGRAPHYKEY,NAMESTYLE
0,21768,2011-01-10,1,19,3578.2700,20110105,3578.2700,310,SO43697,3578.2700,...,None,S,Licenciatura,5,Watson,70000.0,Cole,Direction,53,0
1,28389,2011-01-10,1,39,3399.9900,20110105,3399.9900,346,SO43698,3399.9900,...,None,S,Educación secundaria,3,Martinez,20000.0,Rachael,Ouvrier,222,0
2,25863,2011-01-10,1,100,3399.9900,20110105,3399.9900,346,SO43699,3399.9900,...,None,S,Educación secundaria,5,Wright,40000.0,Sydney,Cadre,543,0
3,14501,2011-01-10,1,100,699.0982,20110105,699.0982,336,SO43700,699.0982,...,None,M,Estudios de postgrado,4,Prasad,80000.0,Ruben,Direction,300,0
4,11003,2011-01-10,1,6,3399.9900,20110105,3399.9900,346,SO43701,3399.9900,...,None,S,Licenciatura,0,Zhu,70000.0,Christy,Cadre,11,0
5,27645,2011-01-11,1,100,3578.2700,20110106,3578.2700,311,SO43702,3578.2700,...,None,M,Educación secundaria,1,Anand,120000.0,Colin,Cadre,312,0
6,16624,2011-01-11,1,6,3578.2700,20110106,3578.2700,310,SO43703,3578.2700,...,None,S,Licenciatura,0,Alvarez,90000.0,Albert,Cadre,30,0
7,11005,2011-01-11,1,6,3374.9900,20110106,3374.9900,351,SO43704,3374.9900,...,None,S,Licenciatura,0,Ruiz,70000.0,Julio,Cadre,22,0
8,11011,2011-01-11,1,6,3399.9900,20110106,3399.9900,344,SO43705,3399.9900,...,None,M,Licenciatura,4,Lu,60000.0,Curtis,Cadre,22,0
9,27621,2011-01-12,1,100,3578.2700,20110107,3578.2700,312,SO43706,3578.2700,...,None,S,Estudios de postgrado,0,Brown,160000.0,Edward,Direction,369,0


<h2>Your Turn...</h2>

In [ ]:
t1 = t0.transform(
    transform_name='',
    )